<a href="https://colab.research.google.com/github/comasya/EntregaFinal_Comas/blob/main/Entrega_Final_Comas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Optimización del stock mínimo de repuestos**

##IA: Generación de prompts

Febrero 2025
Realizado por: Yamil Comas
Comisión 71950

#**Descripción del requerimiento**

El proyecto se basa en una constructora que cuenta con un taller interno donde se realizan las reparaciones y servicios de vehículos livianos y pesados utilizados en las obras. La gestión de repuestos en este taller es un factor crítico para evitar demoras en los servicios y asegurar la operatividad de la flota.

Actualmente, uno de los principales desafíos es identificar de manera eficaz la alta rotación de repuestos/ fluidos y establecer un stock mínimo adecuado en depósito.
Los servicios de mantenimiento tienen intervalos establecidos:

* Vehículos y camiones: cada 10,000 km.
* Moto soldadoras, electro soldadoras, retroexcavadoras, niveladoras y tiende tubos: cada 250 horas de uso.
* Si un equipo ha estado parado por más de 6 meses, se realiza un servicio completo(filtros y fluido).

La falta de un sistema predictivo eficiente puede generar problemas de desabastecimiento, retrasos en los servicios y costos adicionales en compras urgentes.


#**Desarrollo del objetivo**

Se propone la utilización de prompts para el análisis del stock mínimo de repuestos, basándose en la rotación y los servicios programados de los equipos. La solución se dividirá en tres partes principales:
Análisis de repuestos:

1.   Identificación de la rotación de filtros (aceite, combustible, aire) y otros repuestos críticos(correas, mangueras de motor)
2.   Análisis de fluidos: Evaluación del consumo y reposición de aceites y líquido refrigerante.
3.   Análisis combinado: Relación entre repuestos y fluidos, identificando correlaciones y patrones de uso.

Los prompts serán diseñados para generar informes cada 60 días(según necesidad), permitiendo:

*   Sugerir cantidades mínimas recomendadas según el historial de rotación y solicitudes previas.
*   Clasificar repuestos por urgencia, compatibilidad y disponibilidad. Permitir al usuario elegir el formato de salida (tabla, lista priorizada o gráficos).
*   Incluir la visualización de imágenes de repuestos para facilitar su identificación y la búsqueda de alternativas.


In [ ]:
pip install Groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.9/121.9 kB 3.1 MB/s eta 0:00:00


In [5]:
import pandas as pd

In [6]:
# Leer el archivo Excel
df = pd.read_excel('/content/drive/MyDrive/IA PROMPTS/BD_PROYECTO.xlsx')

In [ ]:
print(df.columns)
print(df.head())

Index(['CODIGO', 'NOMBRE_REPUESTO', 'STOCK_ACTUAL', 'CONSUMO_MENSUAL_PROMEDIO',
       'TIEMPO_REPOSICION', 'CATEGORIA'],
      dtype='object')
   CODIGO                   NOMBRE_REPUESTO  STOCK_ACTUAL  \
0    1001   CORREA  5801378010 IVECO 170E22             6   
1    1002  CORREA 0412021761 KOMATSU PC 200             5   
2    1003               CORREA 10 AV 0806 C             3   
3    1004                 CORREA 10 AV 1020             9   
4    1005                 CORREA 10 AV 1165             0   

   CONSUMO_MENSUAL_PROMEDIO  TIEMPO_REPOSICION CATEGORIA  
0                         3                 30    CORREA  
1                         1                 30    CORREA  
2                         2                 30    CORREA  
3                         2                 30    CORREA  
4                         1                 30    CORREA  


In [ ]:
# 2. Preprocesar los datos
# Verificar que las columnas necesarias estén presentes
columnas_necesarias = {'CODIGO', 'NOMBRE_REPUESTO', 'STOCK_ACTUAL', 'CONSUMO_MENSUAL_PROMEDIO','TIEMPO_REPOSICION', 'CATEGORIA'}
if not columnas_necesarias.issubset(df.columns):
    raise ValueError("El archivo no tiene las columnas requeridas")

In [ ]:
# Asegurar que STOCK_ACTUAL sea numérico
df["STOCK_ACTUAL"] = pd.to_numeric(df["STOCK_ACTUAL"], errors='coerce').fillna(0)

In [ ]:
from groq import Groq

# Inicializar el cliente de Groq con tu API key
cliente = Groq(api_key="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")

In [ ]:
# 3. Definir reglas de stock mínimo (mínimo 1 unidad)
df["STOCK_MÍNIMO"] = 1

In [ ]:
# 4. Construir el prompt para Groq
prompt = """
Dado el siguiente dataset de repuestos y fluidos para mantenimiento de maquinaria pesada y vehículos livianos,
analiza el stock actual y determina qué elementos requieren ajustes en el stock mínimo.

Los datos tienen la siguiente estructura:
ID | NOMBRE_REPUESTO | STOCK_ACTUAL | CATEGORÍA | STOCK_MÍNIMO

Aquí están los primeros 5 elementos:
{}

Requisitos:
1. Identificar elementos con stock inferior al mínimo recomendado.
2. Priorizar categorías críticas como FILTRO, ACEITE, COMBUSTIBLE.
3. Generar recomendaciones en formato de texto y tabla.

Responde en formato estructurado.
""".format(df.head().to_string(index=False))


In [ ]:
# Crear la solicitud para obtener la respuesta de la IA usando Groq
respuesta = cliente.chat.completions.create(
    model="llama3-8b-8192",
    messages=[{"role": "system", "content": "Eres un experto en análisis de stock."},
              {"role": "user", "content": prompt}]
)

# Mostrar la respuesta del modelo
print(respuesta.choices[0].message.content)

**Análisis de Stock**

**Elementos con stock inferior al mínimo recomendado:**

1. Cod. 1005: CORREA 10 AV 1165 con 0 unidades, mientras que el stock mínimo recomendado es 1.
2. Cod. 1003: CORREA 10 AV 0806 C con 3 unidades, mientras que el stock mínimo recomendado es 1.

**Categorías críticas:**

* No hay categorías críticas en el dataset proporcionado. Sin embargo, las categorías con stock muy bajo (como CORREA) requieren atención especial.

**Recomendaciones:**

1. Aumentar el stock de CORREA 10 AV 1165 a al menos 1 unidad.
2. Aumentar el stock de CORREA 10 AV 0806 C a al menos 1 unidad.
3. Considerar un sistema de alerta para cuando el stock de cualquier categoría (especialmente CORREA) baje por debajo de un nivel preestablecido.

**Tabla de recomendaciones:**

| Código | NOMBRE_REPUESTO | STOCK_ACTUAL | STOCK_MÍNIMO | Acción |
| --- | --- | --- | --- | --- |
| 1005 | CORREA 10 AV 1165 | 0 | 1 | Aumentar stock |
| 1003 | CORREA 10 AV 0806 C | 3 | 1 | Aumentar stock |

** Nota:** Es

#Análisis de stock faltante de la categoría Filtro

In [ ]:
# Definir el stock mínimo
STOCK_MINIMO = 1

#Identificar códigos con stock mínimo insuficiente por categoría
def identificar_faltantes_por_categoria(df, categoria):
    filtro = (df['CATEGORIA'] == categoria) & (df['STOCK_ACTUAL'] < STOCK_MINIMO)
    return df[filtro]

#Generar tabla con los faltantes críticos
def generar_tabla_faltantes(df):
    faltantes = df[df['STOCK_ACTUAL'] == 0]
    return faltantes

#Ejemplo de ejecución
categoria_elegida = "FILTRO"  # Puedes cambiarlo dinámicamente
faltantes_categoria = identificar_faltantes_por_categoria(df, categoria_elegida)
faltantes_criticos = generar_tabla_faltantes(df)

display(faltantes_criticos)


,CODIGO,NOMBRE_REPUESTO,STOCK_ACTUAL,CONSUMO_MENSUAL_PROMEDIO,TIEMPO_REPOSICION,CATEGORIA,STOCK_MÍNIMO
4,1005,CORREA 10 AV 1165,0,1,30,CORREA,1
18,1019,CORREA 19X1840 (429554) CAT 561,0,1,30,CORREA,1
39,1040,CORREA 4 VIAS 45 V X 1700,0,1,30,CORREA,1
41,1042,CORREA 5H7448,0,2,30,CORREA,1
56,1057,CORREA 8PK1830 GOOD YEAR,0,1,30,CORREA,1
...,...,...,...,...,...,...,...
1473,2474,AGUA Y SOLUCION ANTICONGELANTE (-21°C) CP THERM,0,1,30,ANTICONGELANTE,1
1481,2482,GAS OIL GRADO 1,0,3,30,GAS OIL,1
1501,2502,LUBRICANTE CARDAN 08201/4-00,0,3,30,LUBRICANTE,1
1504,2505,LUBRICANTE MOBIL POLYREX EM,0,2,30,LUBRICANTE,1


In [ ]:
prompt2 = f"""
Analiza el siguiente listado de repuestos con bajo stock en la categoría {categoria_elegida}.
Genera un listado para poder realizar la compra de los faltantes.
{faltantes_categoria.to_string(index=False)}
"""

# Crear la solicitud para obtener la respuesta de la IA usando Groq
respuesta = cliente.chat.completions.create(
    model="llama3-8b-8192",
    messages=[{"role": "system", "content": "Eres un experto en análisis de stock."},
              {"role": "user", "content": prompt2}]
)
# Mostrar la respuesta del modelo
print(respuesta.choices[0].message.content)

A large list of filters with low stock levels! Let's get to work and create a list for purchasing the filters that are needed.

**Filter List to Purchase:**

Based on the data, the following filters have a low stock level (0) and may need to be purchased:

1. **Filtro 1° de Gasilo 5S7645 Cat D6C** (Code: 1276)
2. **Filtro Aceite 1R0739 Cat XXX XXX** (Code: 1310)
3. **Filtro Aceite 3283655 Cat** (Code: 1320)
...
2273. **Filtro TRAMPA DE AGUA 1290373 Cat** (Code: 2325)

**Total Number of Filters to Purchase:** 59

Before making the purchase, it's a good idea to double-check the stock levels and confirm with the supplier or manufacturer to ensure that the filters are indeed out of stock. Additionally, it may be helpful to prioritize the filters based on their usage and importance to ensure that the most critical ones are restocked first.

If you have any further questions or require assistance with anything else, feel free to ask!


In [ ]:
#Análisis de stock faltante de la categoría Correa

In [ ]:
# Definir el stock mínimo
STOCK_MINIMO = 1

#Identificar códigos con stock mínimo insuficiente por categoría
def identificar_faltantes_por_categoria(df, categoria):
    correa = (df['CATEGORIA'] == categoria) & (df['STOCK_ACTUAL'] < STOCK_MINIMO)
    return df[correa]

#Generar tabla con los faltantes críticos
def generar_tabla_faltantes(df):
    faltantes = df[df['STOCK_ACTUAL'] == 0]
    return faltantes

#Ejemplo de ejecución
categoria_elegida = "CORREA"  # Puedes cambiarlo dinámicamente
faltantes_categoria = identificar_faltantes_por_categoria(df, categoria_elegida)
faltantes_criticos = generar_tabla_faltantes(df)

display(faltantes_criticos)


,CODIGO,NOMBRE_REPUESTO,STOCK_ACTUAL,CONSUMO_MENSUAL_PROMEDIO,TIEMPO_REPOSICION,CATEGORIA,STOCK_MÍNIMO
4,1005,CORREA 10 AV 1165,0,1,30,CORREA,1
18,1019,CORREA 19X1840 (429554) CAT 561,0,1,30,CORREA,1
39,1040,CORREA 4 VIAS 45 V X 1700,0,1,30,CORREA,1
41,1042,CORREA 5H7448,0,2,30,CORREA,1
56,1057,CORREA 8PK1830 GOOD YEAR,0,1,30,CORREA,1
...,...,...,...,...,...,...,...
1473,2474,AGUA Y SOLUCION ANTICONGELANTE (-21°C) CP THERM,0,1,30,ANTICONGELANTE,1
1481,2482,GAS OIL GRADO 1,0,3,30,GAS OIL,1
1501,2502,LUBRICANTE CARDAN 08201/4-00,0,3,30,LUBRICANTE,1
1504,2505,LUBRICANTE MOBIL POLYREX EM,0,2,30,LUBRICANTE,1


In [ ]:
prompt3 = f"""
Analiza el siguiente listado de repuestos con bajo stock en la categoría {categoria_elegida}.
Genera un listado para poder realizar la compra de los faltantes.
{faltantes_categoria.to_string(index=False)}
"""

# Crear la solicitud para obtener la respuesta de la IA usando Groq
respuesta = cliente.chat.completions.create(
    model="llama3-8b-8192",
    messages=[{"role": "system", "content": "Eres un experto en análisis de stock."},
              {"role": "user", "content": prompt3}]
)
# Mostrar la respuesta del modelo
print(respuesta.choices[0].message.content)

Basado en el listado proporcionado, he generado un reporte para realizar la compra de los repuestos con bajo stock en la categoría CORREA. El reporte incluye la siguiente información:

**Listado de repuestos con bajo stock**

| Código | Nombre Repuesto | Stock Actual | Consumo Mensual Promedio | Tiempo de Reposición | Sistema de Requisitos (Stock Mínimo) |
| --- | --- | --- | --- | --- | --- |
| 1005 | CORREA 10 AV | 0 | 1 | 30 | Correa |
| 1019 | CORREA 19X1840 (429554) CAT | 0 | 1 | 30 | Correa |
| 1040 | CORREA 4 VIAS 45 V X 1700 | 0 | 1 | 30 | Correa |
| 1042 | CORREA 5H7448 | 0 | 2 | 30 | Correa |
| ... | ... | ... | ... | ... | ... |
| 1268 | CORREA VENTILADOR AV10X0875 DUNLOP | 0 | 3 | 30 | Correa |

**Requisitos de stock para cada repuesto**

* Cod. 1005: 1 unidades (consumo mensual promedio: 1, tiempo de reposición: 30 días)
* Cod. 1019: 1 unidades (consumo mensual promedio: 1, tiempo de reposición: 30 días)
* Cod. 1040: 1 unidades (consumo mensual promedio: 1, tiempo de repos

In [ ]:
!pip install gradio pandas plotly

In [7]:
df["FALTANTE"] = df["STOCK_ACTUAL"] == 0

# Filtrar los repuestos que faltan
faltantes_df = df[df["FALTANTE"]]


In [5]:
import pandas as pd
import plotly.express as px
import gradio as gr

# Leer el archivo Excel
df = pd.read_excel('/content/drive/MyDrive/IA PROMPTS/BD_PROYECTO.xlsx')

# Agregar columna "FALTANTE"
df["FALTANTE"] = df["STOCK_ACTUAL"] == 0
faltantes_df = df[df["FALTANTE"]]
# Agregar columna "STOCK_MINIMO"
df["STOCK_MINIMO"] = df["STOCK_ACTUAL"] == 2
minimos_df = df[df["STOCK_MINIMO"]]

# 📊 Gráfico 1: Cantidad de productos con stock faltante por categoría
def plot_faltantes():
    faltantes_por_categoria = faltantes_df["CATEGORIA"].value_counts().reset_index()
    faltantes_por_categoria.columns = ["CATEGORIA", "CANTIDAD_FALTANTE"]

    fig = px.bar(faltantes_por_categoria, x="CATEGORIA", y="CANTIDAD_FALTANTE",
                 title="Cantidad de Productos con Stock Faltante por Categoría",
                 labels={"CANTIDAD_FALTANTE": "Cantidad de Productos"},
                 color="CATEGORIA",
                 text="CANTIDAD_FALTANTE")
    fig.update_traces(textposition="outside")  # Muestra los valores encima de las barras
    return fig

# 📉 Gráfico 2: Cantidad de productos con stock mínimo por categoría
def plot_stock_minimo():
    minimos_por_categoria = minimos_df["CATEGORIA"].value_counts().reset_index()
    minimos_por_categoria.columns = ["CATEGORIA", "CANTIDAD_MINIMA"]

    fig = px.bar(minimos_por_categoria, x="CATEGORIA", y="CANTIDAD_MINIMA",
                 title="Cantidad de Productos con Stock Mínimo (≤2) por Categoría",
                 labels={"CANTIDAD_MINIMA": "Cantidad de Productos"},
                 color="CATEGORIA",
                 text="CANTIDAD_MINIMA")
    fig.update_traces(textposition="outside")  # Muestra los valores encima de las barras
    return fig

# 🖥️ Crear la interfaz con Gradio
with gr.Blocks() as dashboard:
    gr.Markdown("# 📊 Dashboard de Stock de Repuestos")

    with gr.Row():
        gr.Markdown("### 🔴 Cantidad de Productos con Stock Faltante por Categoría")
        gr.Plot(plot_faltantes)

    with gr.Row():
        gr.Markdown("### ⚠️ Cantidad de Productos con Stock Mínimo (≤2) por Categoría")
        gr.Plot(plot_stock_minimo)

# 🚀 Ejecutar la aplicación
dashboard.launch(share=True)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://8df1a5a87c3208225e.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


#CONCLUSIÓN

El proyecto permite realizar consultas sobre repuestos críticos, identificando de manera eficiente aquellos con stock mínimo (≤2 unidades) y los que están en falta. Esto facilita la planificación de reposición, optimizando los pedidos al sector correspondiente y reduciendo demoras en los servicios. Además, mediante el análisis de rotación y tiempo de reposición, el sistema ayuda a anticipar necesidades, asegurando la disponibilidad de repuestos esenciales y mejorando la gestión del stock de manera proactiva.